# **Estimate the day-ahead merit-order effect of renewable energy for Sweden**

#### **Setting Libraries**

In [1]:
pip install statsmodels --upgrade --pre

Note: you may need to restart the kernel to use updated packages.


In [22]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
import statsmodels.tsa.api as smt

#### **Loading the data**

In [23]:
# Load the pre_panel dataframe
pre_panel = pd.read_csv('pre_panel.csv')

# Sorting by 'entity' and 'time' in descending order
pre_panel = pre_panel.sort_values(by=['subject_id', 'date'], ascending=[True, True])

# Convert the date column to datetime format
pre_panel['date'] = pd.to_datetime(pre_panel['date'], utc=True, format='%Y-%m-%d %H:%M:%S')

#### **Cleaning Database and Creating Dummy and First Difference Variables**

In [24]:
# Rename 'subject_id' to 'entity' and 'date' to 'time'
pre_panel = pre_panel.rename(columns={'subject_id': 'zona', 'date': 'fecha'})

# Create 'dum_ss' and 'dum_aw'
pre_panel['dum_ss'] = pre_panel['mes'].isin([3, 4, 5, 6, 7, 8]).astype(int)
pre_panel['dum_aw'] = pre_panel['mes'].isin([9, 10, 11, 12, 1, 2]).astype(int)

# Create 'dum_wd' and 'dum_we'
pre_panel['dum_wd'] = pre_panel['wd'].isin([1, 2, 3, 4]).astype(int)
pre_panel['dum_we'] = pre_panel['wd'].isin([5, 6, 7]).astype(int)

In [26]:
def calculate_first_differences(df, col_list):
    for col in col_list:
        df[col + "_diff"] = df[col] - df[col].shift(periods=168)

zonas = pre_panel["zona"].unique()
for zona in zonas:
    sub_df = pre_panel[pre_panel["zona"] == zona]
    calculate_first_differences(sub_df, ["price", "hydro", "solar", "wind", "other", "load_var"])
    if zona == 1:
        panel_z1 = sub_df
    elif zona == 2:
        panel_z2 = sub_df
    elif zona == 3:
        panel_z3 = sub_df
    elif zona == 4:
        panel_z4 = sub_df

/var/folders/0j/w3qt49pj7d19pfkd8ry8bfsh0000gn/T/ipykernel_41282/285811854.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col + "_diff"] = df[col] - df[col].shift(periods=168)
/var/folders/0j/w3qt49pj7d19pfkd8ry8bfsh0000gn/T/ipykernel_41282/285811854.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col + "_diff"] = df[col] - df[col].shift(periods=168)
/var/folders/0j/w3qt49pj7d19pfkd8ry8bfsh0000gn/T/ipykernel_41282/285811854.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [27]:
# Concat the DataFrames in the desired order
panel_full = pd.concat([panel_z1, panel_z2, panel_z3, panel_z4], axis=0, ignore_index=True)

#### **Creating the Panel**

In [28]:
# Create a definite Pane
panel = panel_full
# Check for missing values in pre_panel
missing_values = panel.isnull().sum()
# Display the columns with missing values, if any
print("Columns with missing values:")
print(missing_values[missing_values > 0])

Columns with missing values:
load_a              8
hydro              16
solar             192
wind              192
other              16
load_var            8
hydro_diff        700
solar_diff       1056
wind_diff        1056
other_diff        700
load_var_diff     684
dtype: int64


In [6]:
# Dropping Missing Values
panel = panel.dropna()
# Rename 'subject_id' to 'entity' and 'date' to 'time'
panel = panel.rename(columns={'zona': 'id', 'fecha': 'time'})
# Generating External Index
time_index = pd.Index(pre_panel['fecha'])
entity_index = pd.Index(pre_panel['zona'])

In [7]:
# Seting Multindex
panel.set_index(['time','id'], inplace=True)

## **Descriptive Statistics**

#### **Summary Statistics**

In [30]:
# Assuming 'pre_panel' is your DataFrame
vars = ['price', 'hydro', 'solar', 'wind', 'other', 'load_var']

# Calculate mean, standard deviation, min, 25%, 50%, 75%, and max
mean_values = panel_full[vars].apply(np.mean)
std_dev_values =panel_full[vars].apply(np.std)
min_values = panel_full[vars].min()
q25_values = panel_full[vars].quantile(0.25)
median_values = panel_full[vars].median()
q75_values = panel_full[vars].quantile(0.75)
max_values = panel_full[vars].max()

# Create a DataFrame to display the summary statistics
summary_stats = pd.DataFrame({
    'Mean': mean_values,
    'Std Dev': std_dev_values,
    'Min': min_values,
    '25%': q25_values,
    'Median': median_values,
    '75%': q75_values,
    'Max': max_values
})

print(summary_stats)

                 Mean      Std Dev      Min     25%   Median        75%  \
price       75.173247    93.675680   -60.04   17.27    41.17    96.0025   
hydro     1919.621601  1690.214337     6.00  297.00  1433.00  3152.0000   
solar       31.834021    85.070117     0.00    0.00     0.00    10.0000   
wind       961.081585   873.290928     3.00  313.00   716.00  1331.0000   
other      222.446578   275.697313     0.00   41.00   126.00   255.0000   
load_var    45.253517   234.750108 -6441.00  -77.00    37.00   162.0000   

              Max  
price      799.97  
hydro     6577.00  
solar      745.00  
wind      7017.00  
other     1463.00  
load_var  1734.00  


In [ ]:
# Assuming 'pre_panel' is your DataFrame
vars = ['price_diff', 'hydro_diff', 'solar_diff', 'wind_diff', 'other_diff', 'load_var_diff']

# Calculate mean, standard deviation, min, 25%, 50%, 75%, and max
mean_values = panel_full[vars].apply(np.mean)
std_dev_values =panel_full[vars].apply(np.std)
min_values = panel_full[vars].min()
q25_values = panel_full[vars].quantile(0.25)
median_values = panel_full[vars].median()
q75_values = panel_full[vars].quantile(0.75)
max_values = panel_full[vars].max()

# Create a DataFrame to display the summary statistics
summary_stats_d = pd.DataFrame({
    'Mean': mean_values,
    'Std Dev': std_dev_values,
    'Min': min_values,
    '25%': q25_values,
    'Median': median_values,
    '75%': q75_values,
    'Max': max_values
})

print(summary_stats_d)

#### **Correlation Matrix**

In [9]:
correlation_matrix = panel[['price_diff', 'hydro_diff', 'solar_diff', 'wind_diff', 'other_diff', 'load_var_diff']].corr()
print(correlation_matrix)

             price     hydro     solar      wind     other  load_var
price     1.000000 -0.114776  0.050319 -0.309953  0.299960 -0.073367
hydro    -0.114776  1.000000 -0.239845  0.174893 -0.241851  0.064977
solar     0.050319 -0.239845  1.000000 -0.147548  0.150846  0.048050
wind     -0.309953  0.174893 -0.147548  1.000000  0.038418  0.151586
other     0.299960 -0.241851  0.150846  0.038418  1.000000 -0.024690
load_var -0.073367  0.064977  0.048050  0.151586 -0.024690  1.000000


In [ ]:
correlation_matrix = panel[['price', 'hydro', 'solar', 'wind', 'other', 'load_var']].corr()
print(correlation_matrix)

#### **Scatter Plott**

In [ ]:
# Create separate scatter plots for each variable
for var in ['load_var', 'hydro', 'solar', 'wind', 'other']:
    # Create a scatter plot with regression line
    sns.regplot(x=var, y='price', data=panel)
    plt.title(f'Price vs. {var}')
    plt.xlabel(var)
    plt.ylabel('Price')
    plt.show()

In [10]:
panel['intercept'] = 1

# Estimate the two-way fixed effect model
model = sm.OLS(panel['price'], panel[['intercept', 'load_var', 'hydro', 'solar', 'wind', 'other']])
results = model.fit(index=time_index, groups=entity_index, use_demeaned=True)

# Print the results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.196
Model:                            OLS   Adj. R-squared:                  0.196
Method:                 Least Squares   F-statistic:                     3239.
Date:                Tue, 19 Dec 2023   Prob (F-statistic):               0.00
Time:                        00:57:28   Log-Likelihood:            -3.8856e+05
No. Observations:               66521   AIC:                         7.771e+05
Df Residuals:                   66515   BIC:                         7.772e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     84.6982      0.696    121.618      0.0

In [11]:
# Fixed effects model with entity and time effects
sm.OLS(panel['price'], panel[['load_var', 'hydro', 'solar', 'wind', 'other']])
FE_results = model.fit(index=time_index, groups=entity_index)
print('Fixed Effects Model:')
print(FE_results.summary())

Fixed Effects Model:
                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.196
Model:                            OLS   Adj. R-squared:                  0.196
Method:                 Least Squares   F-statistic:                     3239.
Date:                Tue, 19 Dec 2023   Prob (F-statistic):               0.00
Time:                        00:57:28   Log-Likelihood:            -3.8856e+05
No. Observations:               66521   AIC:                         7.771e+05
Df Residuals:                   66515   BIC:                         7.772e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     84.6982      0.69